In [ ]:
import os
import shutil
import glob
import csv
import cv2
import scipy

from seg_functions_TC import*

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
from PIL import Image, ImageDraw, ImageEnhance
from scipy.spatial import ConvexHull
from skimage import measure
import import_ipynb
import find_p2 as p2

In [ ]:
slice = 150

In [ ]:
MAIN_PATH = r'D:\TeamChallengedataTUe\Team challenge 2021\Scoliose'
cso_file = os.path.join(MAIN_PATH,'Scoliose/1postop.cso') 
nifti_file = os.path.join(MAIN_PATH,'Scoliose/1postop.nii')

ct_img = nib.load(nifti_file)
pixdim = find_pix_dim(ct_img)
ct_numpy_all = ct_img.get_fdata()

ct_numpy = ct_numpy_all[:,:,slice]
contours = intensity_seg(ct_numpy, min=-1000, max=-300) # Hounsfields units
lungs = find_lungs(contours)

if lungs:
    lung_mask = create_mask_from_polygon(ct_numpy, lungs)

In [ ]:

# Plot the images
fig, ax = plt.subplots(1, 2, figsize=(20, 20))
ax[0].imshow(ct_numpy, cmap='gray')
ax[0].set_title('Input image')
ax[1].imshow(lung_mask, cmap='gray')
ax[1].set_title('Segmented lungs')
    
[x.set_axis_off() for x in ax]
plt.show()



In [ ]:
bone_threshold_filtered_image = p2.filtered_images(ct_numpy)
lung_mask_90, l1, l2 = p2.find_most_anterior_lung_points(lung_mask)
posterior_centroid_sternum_row, col_centroid_sternum = p2.centroid_sternum(lung_mask_90, bone_threshold_filtered_image, l1, l2)

In [ ]:
#all operations below this cell are implemented in the used functions from find_p2 in this cell

rot90_im = np.rot90(ct_numpy)
imgplot = plt.imshow(rot90_im,cmap = 'gray')
plt.plot(col_centroid_sternum, posterior_centroid_sternum_row, marker='o', markersize=5, color="red")

In [ ]:
ct_numpy = ct_numpy_all[:,:,slice]
contours_bone = intensity_seg(ct_numpy, min=100, max=300) # Hounsfields units
bones = create_mask_from_polygon(ct_numpy, contours_bone)

kernel = np.ones((5,5),np.uint8)
gradient = cv2.morphologyEx(bones, cv2.MORPH_GRADIENT, kernel)

# filter = ImageEnhance.Contrast(bones)
# bones.filter(1.5)

bones_filt = cv2.GaussianBlur(bones,(3,3),0)
laplacian = cv2.Laplacian(bones_filt,cv2.CV_64F)


# # Plot the images
# fig, ax = plt.subplots(1, 3, figsize=(20, 20))
# ax[0].imshow(ct_numpy, cmap='gray')
# ax[0].set_title('Input image')
# ax[1].imshow(gradient, cmap='gray')
# ax[1].set_title('bone')
# ax[2].imshow(laplacian, cmap='gray')
# ax[2].set_title('laplacian')
# plt.show()

In [ ]:
plt.hist(ct_numpy.ravel(), 4000, [-1000,1000], color='crimson')
plt.ylabel("Number Of Pixels", color='crimson')
plt.xlabel("Pixel Intensity- From 0-255", color='crimson')
plt.title("Histogram Showing Pixel Intensity And Corresponding Number Of Pixels", color='crimson')
plt.show()

In [ ]:
HUbone = 200 #bone is around 300-400 HU (taking artefacts such as streaking- and cupping artefacts in account so decreasing to 200)
ret,thresh = cv2.threshold(ct_numpy,HUbone,3000,cv2.THRESH_BINARY)

In [ ]:
thres = thresh.astype('int16')# om te voorkomen dat je een of andere vage error over orde van grootte van de getallen krijgt
bones_thresh_filt = cv2.medianBlur(thres, 3)

im  = (lung_mask*3000)+(bones_thresh_filt) #(overplotted)



# Plot the images
fig, ax = plt.subplots(1, 4, figsize=(20, 20))
ax[0].imshow(ct_numpy, cmap='gray')
ax[0].set_title('Input image')
ax[1].imshow(thresh, cmap='gray')
ax[1].set_title('threshold')
ax[2].imshow(bones_thresh_filt, cmap='gray')
ax[2].set_title('threshold filter')
ax[3].imshow(im, cmap='gray')
ax[3].set_title('CT + mask+bones')

plt.show()

print(format(lung_mask.max()))
print(format(lung_mask.min()))

In [ ]:
# find most anterior points in lung volumes

im90 = np.rot90(lung_mask)
arr = np.nonzero(im90)

# kernel = np.ones((10,10),np.uint8)
# opening = cv2.morphologyEx(im90, cv2.MORPH_GRADIENT, kernel)

l1 = [arr[0][0]],[arr[1][0]] #eerste punt (arr[rijnummer][kolomnummer]

# Marker labelling to to able to distinct between masks
ret, markers = cv2.connectedComponents(im90)

# één masker verwijderen
markers[markers == markers[l1]] = 0 # markers in markers die ongelijk zijn aan de waarde van het eerste punt 
arr2 = np.nonzero(markers)
l2 = [arr2[0][0]],[arr2[1][0]] 
# [arr2[as (0=rij of 1=kolom)][hoeveelste indices]],[arr2[as (0=rij of 1=kolom)][hoeveelste indices]] !!!

imgplot = plt.imshow(im90, cmap='gray')
plt.plot(l1[1], l1[0], marker='o', markersize=5, color="red")
plt.plot(l2[1], l2[0], marker='o', markersize=5, color="blue")

col_l1, _  = l1[1], l1[0]
col_l2, _  = l2[1], l2[0]
fov_between_lungs = l1[1], l2[1]
fov_between_lungs = np.sort(fov_between_lungs, axis=None)
rows = np.prod(im90[1].shape)
print(fov_between_lungs[1])

In [ ]:
# set values of columns before the first point and points after column of the second point to zero (of rotated original image)

bones_thresh_filt_90 = np.rot90(bones_thresh_filt)
lung_mask_90 = np.rot90(lung_mask)
mask = np.zeros(lung_mask_90.shape)
mask[l1[0][0]-50:l1[0][0]+50,fov_between_lungs[0]:fov_between_lungs[1]]=1

middle_col = np.around((fov_between_lungs[0]+fov_between_lungs[1])/2)
# middle_row = np.around()
print(fov_between_lungs[0],fov_between_lungs[1])
fov_im90 = bones_thresh_filt_90*mask
imgplot = plt.imshow(fov_im90, cmap='gray')
# plt.plot(middle_point, 140, marker='o', markersize=5, color="blue") # middle of the two lung points

print(l1[0][0])

In [ ]:
# posterior middelpunt sternum bepalen:

#closing uitvoeren
kernel = np.ones((5,5),np.uint8)
closing = cv2.morphologyEx(fov_im90, cv2.MORPH_CLOSE, kernel, iterations=3)
closing = closing.astype('int8')# om te voorkomen dat je een of andere vage error over orde van grootte van de getallen krijgt
imgplot = plt.imshow(closing, cmap='gray')


# Choose 4 or 8 for connectivity type
connectivity = 4  
masks = cv2.connectedComponentsWithStats(closing, connectivity, cv2.CV_32S)

num_labels = masks[0]
stats = masks[2]
centroids = masks[3]

for label in range(1,num_labels):
    blob_area = stats[label, cv2.CC_STAT_AREA]
    blob_width = stats[label, cv2.CC_STAT_WIDTH]
    blob_height = stats[label, cv2.CC_STAT_HEIGHT]

mask_area = stats[:,4]
filtered_mask_area = np.sort([x for x in mask_area if x <= 100000 and x > 50])
a = stats[:,4] == filtered_mask_area[0]
b=[i for i, x in enumerate(a) if x]
centroid_sternum = np.around(centroids[b])

# find lowest nonzero point in mask

    
# ind=np.argsort(stats[:,-1])
# stats[ind]
# stats = np.sort(stats[:,])    
# print(filtered_mask_area)
# print(centroid_sternum)
# print(centroid_sternum[0][0])
# print(centroids)

#ret,markers = cv2.connectedComponents(closing)


#find lowest point of mask in centroid
imgplot = plt.imshow(closing, cmap='gray')


In [ ]:

row_centroid_sternum = np.uint(centroid_sternum[0][1])
col_centroid_sternum = np.uint(centroid_sternum[0][0])
centroid_axis = closing[:,col_centroid_sternum]
centroid_axis_corr = centroid_axis[row_centroid_sternum:]
# print(row_centroid_sternum)
# print(centroid_axis)
index_backgroud_mask = np.where(centroid_axis_corr[:-1] != centroid_axis_corr[1:])[0] # find indices where the value changes.
# add the value of the first index to the centroid row value
posterior_centroid_sternum_row = index_backgroud_mask[0]+row_centroid_sternum
print(posterior_centroid_sternum_row)

rot90_im = np.rot90(ct_numpy)

imgplot = plt.imshow(rot90_im,cmap = 'gray')
plt.plot(col_centroid_sternum, posterior_centroid_sternum_row, marker='o', markersize=5, color="red")

In [ ]:
# sternum niet altijd zichtbaar, in dit geval middelpunt pakken van de meest anterior bepaalde punten van de longvolumes?





## bouw een functie in die het volgende doet om het sternum te detecteren: 
    # de botstructuren detecteren
    # een voorwaarde inbouwen om te kijken of deze botten (vrijwel) direct bij de longen liggen
    # een voorwaarde inbouwen om te kijken of deze dichtbij het hart ligt (zacht weefsel)
    # als aan beide voorwaarden voldaan worden dan kan het detectie punt voor p2 worden bepaald. 
    
    #- mocht die niet lukken dan kan gekeken worden naar het frontale vlak om het sternum aan te duiden en zo weer in het axiale vlak laten zien.
    # deze is hier prominenter aanwezig
    
  

In [ ]:

    #find bony structures that are between the lung masks
    # if none of no lung masks are present return that none could be found or no lungs are present
    #assume the heart is in every case behind the sternum
    # return the image area between the two most anterior lung points set the the rest to zero
    

In [ ]:
 # geeft geen goed beeld van het sternum in frontaal en sagittaal vlak
    
# ct_numpy_f = ct_numpy_all[260,:,:]
# ct_numpy_f = np.rot90(ct_numpy_f)
# fig, ax = plt.subplots(1, 2, figsize=(20, 20))
# ax[0].imshow(ct_numpy_f, cmap='gray')
# ax[0].set_title('Input image')
# ax[0].imshow(ct_numpy_f, cmap='gray')
# ax[0].set_title('Input image')

In [ ]:
# find point 7